# 2.5.1 Agent의 히스토리를 관리하는 방법
- 에이전트의 히스토리를 관리해서 대화를 이어나가는 방법을 알아봅니다
- 히스토리를 관리를 위해 `checkpointer`를 사용합니다.
- `checkpointer`는 두 가지 방법을 제공합니다
    - 메모리에 저장하는 방법
    - 데이터베이스에 저장하는 방법
- 메모리에 저장하는 방법을 알아봅니다
    - 다양한 [`checkpointer`](https://langchain-ai.github.io/langgraph/concepts/persistence/#checkpointer-libraries)를 확인해보세요
  


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [3]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """숫자 a와 b를 더합니다."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """숫자 a와 b를 곱합니다."""
    return a * b

In [4]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

In [5]:
from langchain_google_community import GmailToolkit

from langchain_google_community.gmail.utils import (
    build_resource_service,
    get_gmail_credentials,
)

# Can review scopes here https://developers.google.com/gmail/api/auth/scopes
# For instance, readonly scope is 'https://www.googleapis.com/auth/gmail.readonly'
credentials = get_gmail_credentials(
    token_file="./gmail_token.json",
    scopes=["https://mail.google.com/"],
    client_sercret_file="./gmail_credentials.json",
)
api_resource = build_resource_service(credentials=credentials)
gmail_toolkit = GmailToolkit(api_resource=api_resource)
gmail_tool_list = gmail_toolkit.get_tools()


/var/folders/qt/w7p0s8cx51q4dh533wlpsbrm0000gn/T/ipykernel_15911/4225151943.py:10: DeprecationWarning: get_gmail_credentials is deprecated and will be removed in a future version.Use get_google_credentials instead.
  credentials = get_gmail_credentials(
/var/folders/qt/w7p0s8cx51q4dh533wlpsbrm0000gn/T/ipykernel_15911/4225151943.py:15: DeprecationWarning: build_resource_service is deprecated and will be removed in a future version.Use build_gmail_service instead.
  api_resource = build_resource_service(credentials=credentials)


In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools.retriever import create_retriever_tool

embedding_function = OpenAIEmbeddings(model='text-embedding-3-large')

vector_store = Chroma(
    embedding_function=embedding_function,
    collection_name = 'chroma_collection',
    persist_directory = './chroma_collection'
)
retriever = vector_store.as_retriever(search_kwargs={'k': 3})
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name='young_housing_support_retriever',
    description='Contains information about 청년월세지원 2025',
)

In [7]:
from langgraph.prebuilt import ToolNode

tool_list = [add, multiply, search_tool, retriever_tool] + gmail_tool_list # + loaded_tool_list
llm_with_tools = llm.bind_tools(tool_list)
tool_node = ToolNode(tool_list)

In [8]:
from langgraph.graph import MessagesState, StateGraph

graph_builder = StateGraph(MessagesState)

In [9]:
def agent(state: MessagesState) -> MessagesState:
    messages = state['messages']
    response = llm_with_tools.invoke(messages)
    return {'messages': [response]}

In [ ]:
graph_builder.add_node('agent', agent)
graph_builder.add_node('tools', tool_node)

In [ ]:
from langgraph.graph import START, END
from langgraph.prebuilt import tools_condition


graph_builder.add_edge(START, 'agent')
graph_builder.add_conditional_edges(
    'agent',
    tools_condition
)
graph_builder.add_edge('tools', 'agent')

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

checkpointer = MemorySaver()
graph= graph_builder.compile(checkpointer=checkpointer)

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage

config = {
    'configurable': {
        'thread_id': 'search_and_summarize'
    }
}

query = 'seungju41625886@gmail.com으로 MCP에 대해서 검색한 내용을 200자 내로 요약해서 메일의 초안을 작성해주세요.'
for chunk in graph.stream({'messages': [HumanMessage(query)]}, stream_mode='values', config=config):
    chunk['messages'][-1].pretty_print()

In [ ]:
config = {
    'configurable': {
        'thread_id': 'search_and_summarize'
    }
}

query = '메일 초안 마지막에 "좋은 하루 보내세요"라고 추가하고 메일을 전송해주세요.'
for chunk in graph.stream({'messages': [HumanMessage(query)]}, stream_mode='values', config=config):
    chunk['messages'][-1].pretty_print()